# LLM-powered Screening at Scale

The everyrow `screen()` function filters a dataframe by applying LLMs, and LLM research agents, to every row to determine if the criteria are met. This notebook demonstrates how this scales to screening 10,000 rows. Since tricky rows get LLM agents that themselves make dozens of LLM calls, this results in running vastly more LLM calls than is generally feasible without dedicated orchestration or infrastructure. The total cost is ~$0.001 per row.

## Example: Filtering 10,000 FDA Recalls

This example takes a dataset of FDA product recalls and filters it to find recalls relevant to a specific personal situation: products that might have been used for a child born on a particular date. The screening task requires understanding product types, typical use cases, and timing to determine which recalls matter.

## Load Data

In [ ]:
# !pip install everyrow
import pandas as pd
from everyrow import create_session
from everyrow.ops import screen

pd.set_option("display.max_colwidth", None)


# load API key from environment/.env file or set it directly in the notebook
from dotenv import load_dotenv
load_dotenv()
# import os
# os.environ["EVERYROW_API_KEY"] = "get an API key on everyrow.io. $20 free"


In [ ]:
fda_product_recalls_df = pd.read_csv("https://media.githubusercontent.com/media/futuresearch/everyrow-sdk/main/docs/case_studies/llm-powered-screening-at-scale/fda_product_recalls.csv")

# Filter to recalls where center_classification_date is after 2021-08-01 to get a dataset with ≈10k rows
fda_product_recalls_df["center_classification_date"] = pd.to_datetime(fda_product_recalls_df["center_classification_date"], errors="coerce")
fda_product_recalls_df = fda_product_recalls_df[fda_product_recalls_df["center_classification_date"] > pd.Timestamp("2021-08-01")]

print(len(fda_product_recalls_df))
fda_product_recalls_df.sample(3)

9949


,fei_number,recalling_firm_name,product_type,product_classification,status,distribution_pattern,recalling_firm_city,recalling_firm_state,recalling_firm_country,center_classification_date,reason_for_recall,product_description,event_id,event_classification,product_id,center,recall_details
7969,1.970356e+06,Mississippi Valley Regional Blood Center,Biologics,Class III,Terminated,Missouri; Illinois,Davenport,Iowa,United States,2021-12-09,"Blood products, labeled with incorrect volumes, were distributed.\r\n","Apheresis Red Blood Cells Deglycerolized, Leukocytes Reduced",89025,Class III,190301,CBER,https://www.accessdata.fda.gov/scripts/ires/?Product=190301
3011,1.220592e+06,"Hobbs Medical, Inc.",Devices,Class II,Ongoing,"Worldwide distribution - US Nationwide and the countries of Canada, Austria, Belgium, Finland, Norway.",Stafford Springs,Connecticut,United States,2022-10-07,"Device label display an incorrect expiration date, extends the shelf life of the product. Affected product distributed in the United States from 3/25/2018 to 9/28/2020, expiration dates between 2022-11-01 and 2025-09-01 (Devices have a lot number ending in R, or are kits containing at least 1 device with a lot number ending in R.)",Hobbs\tBiliary Pigtail Stent Kits (contains 3412)\r\n\r\nCatalog Number: 6027 \r\n\r\n\r\n \r\n\r\n\r\n\r\n\r\n\r\n\t\r\n\r\n\r\n\r\n\r\n\r\n,90769,Class II,195819,CDRH,https://www.accessdata.fda.gov/scripts/ires/?Product=195819
9683,3.016258e+09,CB DISTRIBUTION LLC,Food/Cosmetics,Class III,Terminated,Product was shipped to distribution centers in Texas,Waynesboro,Mississippi,United States,2021-08-17,Incorrect Ingredient Labeling,Cotton Blues Cheesecake Co.\nStrawberry Swirl Single Slice\n6oz plastic container\n36 packages per case,88458,Class III,188714,CFSAN,https://www.accessdata.fda.gov/scripts/ires/?Product=188714


## Define Screen Task

The screening criteria specify finding recalls of products that might have been used for a child born on 2021-08-01.

In [8]:
SCREEN_TASK = """Find recalls of products that I might have used for my child born on 2021-08-01."""

## Run Screen

In [ ]:
async def run_screen():
    async with create_session(name="FDA Recall Screening Workflow") as session:
        print(f"Session URL: {session.get_url()}")
        print("\nScreening FDA recalls...\n")

        result = await screen(
            task=SCREEN_TASK,
            input=fda_product_recalls_df, # type: ignore
        )

        return result.data


results_df = await run_screen()

Session URL: https://everyrow.io/sessions/df145a50-2dfd-48c6-97ed-6f82a82bca66

### Cost

This run cost $12.10, averaging around $0.001 per row.

## Inspecting results

In [ ]:
results_df = pd.read_csv("https://media.githubusercontent.com/media/futuresearch/everyrow-sdk/main/docs/case_studies/llm-powered-screening-at-scale/Screen%20child%20product%20recalls.csv") # download from https://everyrow.io/sessions/df145a50-2dfd-48c6-97ed-6f82a82bca66

In [14]:
reduced_results_df = results_df[
    [
        "product_type",
        "status",
        "reason_for_recall",
        "research",
        "center_classification_date",
        "product_description",
        "research",
        "screening_result",
    ]
]
yes_samples = reduced_results_df[
    reduced_results_df["screening_result"] == "yes"
].sample(5)
no_samples = reduced_results_df[reduced_results_df["screening_result"] == "no"].sample(
    5
)

In [15]:
yes_samples

,product_type,status,reason_for_recall,research,center_classification_date,product_description,research,screening_result
6245,Devices,Ongoing,Facility Temperature excursions (from June 2021-August 2021) prior to delivery; exposure to higher temperatures may have impacted product effectiveness. All lots received between 06/01/21-9/30/21.,"{""screening_result"":""Nail polish remover pads are commonly used in households. A parent might use these on themselves or occasionally on a child. The recall is due to temperature excursions potentially impacting effectiveness. It is a standard household item a parent of a young child might have.""}",1661385600000,"Dynarex¿\tPAD, NAIL POLISH REMOVER (100/BX 10BX/CS) DYNREX\t\t\r\n\t\r\nCatalog # 1501","{""screening_result"":""Nail polish remover pads are commonly used in households. A parent might use these on themselves or occasionally on a child. The recall is due to temperature excursions potentially impacting effectiveness. It is a standard household item a parent of a young child might have.""}",yes
8625,Drugs,Ongoing,Microbial Contamination of Non-Sterile Products: FDA Laboratory results found product to be contaminated with Bacillus lichenformis and Bacillus sonorensis.,"{""screening_result"":""This first aid kit contains After Burn Cream, which is a common household item a parent might use for a child's minor injuries. The recall due to microbial contamination was classified in January 2023, when the child would have been about 17 months old.""}",1674777600000,"Easy Care First Aid 25 Person 2009 ANSI, First Aid Kit, containing Easy Care first aid After Burn Cream 0.9 g packets, Product # 9999-2132","{""screening_result"":""This first aid kit contains After Burn Cream, which is a common household item a parent might use for a child's minor injuries. The recall due to microbial contamination was classified in January 2023, when the child would have been about 17 months old.""}",yes
4789,Drugs,Ongoing,cGMP deviations: product stored outside of labeled temperature requirements.,"{""screening_result"":""The product is hand sanitizer, classified in May 2022. The child would be 9 months old. Hand sanitizer is a staple for parents to maintain hygiene for themselves and their children, especially when on the go. It is highly likely to be used for or around the child.""}",1653955200000,"Luxury Hygiene Hand Sanitizer Moisturizing Formula with Vitamin E, 8 FL. OZ. (236 mL), TSM Brands LLC 540 Equinox Ln Manalapan, NJ 07726, UPC 6 86275 96536 9","{""screening_result"":""The product is hand sanitizer, classified in May 2022. The child would be 9 months old. Hand sanitizer is a staple for parents to maintain hygiene for themselves and their children, especially when on the go. It is highly likely to be used for or around the child.""}",yes
9413,Food/Cosmetics,Ongoing,"On 02/03/2023, FRESH IDEATION FOOD GROUP, LLC dba Fresh Creative Cuisine of Baltimore, MD is initiating a recall of various products sold from January 24, 2023 through January 30, 2023 because the products have the potential to be contaminated with Listeria monocytogenes .","{""screening_result"":""The product is a breakfast muffin (Bacon, Colby, Cheddar & Egg). At the time of the recall in February 2023, the child would have been approximately 1.5 years old and capable of consuming such food.""}",1678406400000,"Bistro To Go , Bacon, Colby, Cheddar & Egg Muffin, Net . Wt. 6.5 Oz. Single Serving, Mylar Wrap, Keep refrigerated","{""screening_result"":""The product is a breakfast muffin (Bacon, Colby, Cheddar & Egg). At the time of the recall in February 2023, the child would have been approximately 1.5 years old and capable of consuming such food.""}",yes
5946,Drugs,Ongoing,CGMP Deviations: products were stored outside the drug label specifications.,"{""screening_result"":""Systane Balance is an over-the-counter lubricant eye drop. Parents may use lubricating eye drops on children for dry eyes or irritation. The recall occurred in Augus

In [16]:
no_samples

,product_type,status,reason_for_recall,research,center_classification_date,product_description,research,screening_result
519,Devices,Terminated,The sterile barrier seal may be compromised.,"{""screening_result"":""This is a surgical device kit for discectomy (back surgery), which is not a product used by a parent for a child. The recall date was September 2, 2021.""}",1630540800000,"Lase DiscectomLase Discectomy Device Kit, Models 1100-002 and 1100-002 SHORTy Device Kit, Models 1100-002","{""screening_result"":""This is a surgical device kit for discectomy (back surgery), which is not a product used by a parent for a child. The recall date was September 2, 2021.""}",no
9494,Devices,Ongoing,The subject product lots are being recalled because they received a higher than specified irradiation dose. This exceeds the validated range for exposure to gamma radiation of these devices and may result in changes to the implant material properties.,"{""screening_result"":""The Attune Revision LPS Tibial Insert is an orthopedic implant used in revision knee replacement surgeries, typically for adults or patients with significant bone loss. It is not a standard product used for a child born in 2021. The recall was due to excessive irradiation doses [f42c12d6-e9be-5928-b889-82fb1dd94353].""}",1679011200000,"Attune Revision Limb Preservation System (LPS) Tibial Insert XSM 26MM, Part Number 151760226","{""screening_result"":""The Attune Revision LPS Tibial Insert is an orthopedic implant used in revision knee replacement surgeries, typically for adults or patients with significant bone loss. It is not a standard product used for a child born in 2021. The recall was due to excessive irradiation doses [f42c12d6-e9be-5928-b889-82fb1dd94353].""}",no
8529,Drugs,Ongoing,Lack of Assurance of Sterility: FDA inspection revealed insanitary conditions at the facility.,"{""screening_result"":""This is a compounded irrigation solution (Heparin/Lidocaine/Sodium Bicarbonate) used in clinical or surgical settings. It is not a consumer product or a standard pediatric care item used by a parent for a child.""}",1674691200000,"HEPARIN/LIDO HCL/SODIUM BICARB 100U/2MG/16.8MG/ML (20ML SYRINGE) IRRIGATION, Rx Only, Pharmacy Innovations, 2936 W 17th St., Erie, PA. \n","{""screening_result"":""This is a compounded irrigation solution (Heparin/Lidocaine/Sodium Bicarbonate) used in clinical or surgical settings. It is not a consumer product or a standard pediatric care item used by a parent for a child.""}",no
29,Food/Cosmetics,Terminated,May be contaminated with Listeria monocytogenes,"{""screening_result"":""This is a chocolate chip brownie snack cup. The recall occurred in August 2021, when the child was a newborn and would not have been consuming solid snack foods like brownies. [46c57ad0-94ed-5f00-91f9-ba81d1512838]""}",1627948800000,"Great Value Chocolate Chip Brownie Snack Cup - 12 oz., 8 per case; UPC: 078742-33099 0","{""screening_result"":""This is a chocolate chip brownie snack cup. The recall occurred in August 2021, when the child was a newborn and would not have been consuming solid snack foods like brownies. [46c57ad0-94ed-5f00-91f9-ba81d1512838]""}",no
4602,Drugs,Ongoing,Lack of assurance of sterility.,"{""screening_result"":""Ondansetron Hydrochloride 1 mg/mL in a 30 mL multi-dose vial is a prescription medication primarily for clinical or hospital use and is not a common household product for a child.""}",1653436800000,"Ondansetron Hydrochloride, 1 mg/mL, Multi-Dose 30 mL vial, Rx Only, Olympia Pharmaceuticals, 6700 Conroy Rd., Ste. 155 Orlando, FL 32835. NDC 73198-0127-30.","{""screening_result"":""Ondansetron Hydrochloride 1 mg/mL in a 30 mL multi-dose vial is a prescription medication primarily for clinical or hospital use and is not a common household product for a child.""}",no
